In [1]:
# !pip install azure-search-documents==11.4.0b8
# !pip install azure-search
# !pip install langchain=0.0.348
# !pip install openai=0.28

# Azure AI Search LangChain vector code sample
This code demonstrates how to use Azure AI Search with OpenAI and the langchain.vectorstores.azuresearch`` module. To run the code, install the following packages, including th 'eazure-search-documents==11.4.0b'` packaged, as noted in LangChain docs.

## imports and env variables

In [2]:
import os
import openai
# from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv

In [3]:
# Set environment variables
load_dotenv()

True

In [4]:
openai.api_key = os.getenv("api_key")
openai.api_base = os.getenv("api_base")
openai.api_type = os.getenv("api_type")
openai.api_version = os.getenv("api_version")
deployment_id = os.getenv("deployment_id_gpt_4")
gpt_model = os.getenv("deployment_id_gpt_4")
model_engine = os.getenv("deployment_id_gpt_4")
embd_model = 'text-embedding-ada-002'

os.environ.update({
    "OPENAI_API_TYPE": os.getenv("api_type"),
    "OPENAI_API_VERSION": os.getenv("api_version"),
    "OPENAI_API_BASE": os.getenv("api_base"),
    "OPENAI_API_KEY": os.getenv("api_key")
})
model: str = "text-embedding-ada-002"

## Configure vector store settings

In [5]:
service_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
key = os.getenv("AZURE_AI_SEARCH_KEY")
endpoint = "https://{}.search.windows.net/".format(service_name)

vector_store_address: str = endpoint
vector_store_password: str = key

In [6]:
from azure.search.documents.indexes.models import (
    SemanticSettings,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField
)

### Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your search index:

In [7]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key, deployment=embd_model)

index_name: str = "vector-idx-test"


vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name='config',
        semantic_settings=SemanticSettings(
            default_configuration='config',
            configurations=[
                SemanticConfiguration(
                    name='config',
                    prioritized_fields=PrioritizedFields(
                        title_field=SemanticField(field_name='content'),
                        prioritized_content_fields=[SemanticField(field_name='content')],
                        prioritized_keywords_fields=[SemanticField(field_name='metadata')]
                    ))
            ])
    )

### Insert text and embeddings into vector store

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['MDBiNDQ5NTUtMzNmYS00ZDdjLTg3NWEtMjMyMGRmMzJjN2Iw',
 'NjRiNzlhZjUtZWRiMi00MzVmLWIyODItNzEyYWNiZTFhYzU2',
 'M2EwZDYzMDUtOTFmMC00M2I3LWFjNTQtNDcxYzM3NDY0ZWI0',
 'ZmUxOTM5ZjktODlkMS00OTFmLWI1NmMtOTc0NTg5MmY5NjEz',
 'ZDNiOTNkY2QtMTRlMC00YzAwLTgxZWEtMmY0NTYyN2I2MDFh',
 'NTg1YjFiOWEtM2M5OC00NDliLTk5ZjEtOTllZjJmNzBiNDhk',
 'MTM0YmNlMWEtZDRiMy00ZDdjLWEwYmUtYjMzMjY5MzM4ZWYw',
 'OGViZTlhZDQtY2RjNi00Y2I1LWE5ZTItOTJmY2U3YjgxM2Uw',
 'ZjEwNTZjODMtMzI3Zi00ZGNhLTlmODktZTc1NzM4NDI0NGIz',
 'ZWZmZTUxMDUtNmI3Ny00ZGNlLThiM2UtYTQ1NjAwMTA4NDBl',
 'OWI2OTFjOGMtMGRlZC00YjY2LTg3MTktMjk4ZjU4Nzg2NmY2',
 'YWQyOTZhNmUtMTQxYi00NjI1LTk0YzYtOWJkODRiNGJiNjQ5',
 'OTQ0OTNhNDMtMzczNi00OGU0LTllODUtZGE3MzAzMzZhNzI2',
 'NTMyMmE1OTgtN2E4NS00YTMyLTg2NDgtOTE5MTQxMGZlNmFl',
 'NmRiOWUwZmYtZjEwZS00MDcyLTk4MjMtM2RhZmQ5YmYzZTBh',
 'ZDNkMzJiM2YtMDViNy00OTBlLThmZmUtMDIzZDk3ZDBlOGIx',
 'NTBlNzA1OTAtNmExMi00MTM2LWEzNjAtNzQ1OTU0ZDA4MjVk',
 'OTYxNWExOTYtZmE5Zi00MjBmLWEwNmItMTYzYjlmMDYyZTVh',
 'M2M1OTIxMzgtMjExOC00M2UxLTg1YzEtMDFmNTM3MzIy

### Perform a vector similarity search 

In [9]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### Perform a hybrid search

In [10]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3, 
    search_type="hybrid"
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### Perform a hybrid search with semantic reranking (powered by Bing)
Semantic ranking enhances the quality of search results for text-based queries. By enabling semantic ranking on your search service, you can improve the relevance of search results in two ways:

It applies secondary ranking to the initial result set, promoting the most semantically relevant results to the top.
It extracts and returns captions and answers in the response, which can be displayed on a search page to enhance the user's search experience.

In [11]:
# Perform a hybrid search with semantic reranking  
docs_and_scores = vector_store.semantic_hybrid_search_with_score(  
    query="What did the president say about Ketanji Brown Jackson",  
    k=3,  
)  
  
# Print the results  
for doc, score in docs_and_scores:  
    print("-" * 80)  
    answers = doc.metadata['answers']  
    if answers:  
        if answers.get('highlights'):  
            print(f"Semantic Answer: {answers['highlights']}")  
        else:  
            print(f"Semantic Answer: {answers['text']}")  
        print(f"Semantic Answer Score: {score}")  
    print("Content:", doc.page_content)  
    captions = doc.metadata['captions']
    print(f"Score: {score}") 
    if captions:  
        if captions.get('highlights'):  
            print(f"Caption: {captions['highlights']}")  
        else:  
            print(f"Caption: {captions['text']}")  
    else:  
        print("Caption not available")  

--------------------------------------------------------------------------------
Content: Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
Score: 0.03333333507180214
Caption: And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge<em> Ketanji